# Semantic Kernel met OpenBnB MCP Server Integratie

Deze notebook toont hoe je Semantic Kernel gebruikt met de daadwerkelijke OpenBnB MCP-server om te zoeken naar echte Airbnb-accommodaties met behulp van MCPStdioPlugin. Voor LLM-toegang wordt Microsoft Foundry gebruikt. Om je omgevingsvariabelen in te stellen, kun je de [Setup Les](/00-course-setup/README.md) volgen.


## Importeer de benodigde pakketten


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Het maken van de MCP Plugin-verbinding

We maken verbinding met de [OpenBnB MCP-server](https://github.com/openbnb-org/mcp-server-airbnb) met behulp van MCPStdioPlugin. Deze server biedt Airbnb-zoekfunctionaliteit via het @openbnb/mcp-server-airbnb-pakket.


## De client maken

In dit voorbeeld zullen we Microsoft Foundry gebruiken voor onze LLM-toegang. Zorg ervoor dat je omgevingsvariabelen correct zijn ingesteld.


## Omgevingsconfiguratie

Configureer de Azure OpenAI-instellingen. Zorg ervoor dat je de volgende omgevingsvariabelen hebt ingesteld:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Begrijpen van de OpenBnB MCP-integratie

Deze notebook maakt verbinding met de **echte OpenBnB MCP-server** die daadwerkelijke Airbnb-zoekfunctionaliteit biedt.

### Hoe het werkt:

1. **MCPStdioPlugin**: Gebruikt standaard input/output communicatie met de MCP-server
2. **Echt NPM-pakket**: Downloadt en draait `@openbnb/mcp-server-airbnb` via npx
3. **Live Data**: Retourneert werkelijke Airbnb-properties gegevens van hun API's
4. **Functieontdekking**: De agent ontdekt automatisch beschikbare functies van de MCP-server

### Beschikbare functies:

De OpenBnB MCP-server biedt meestal:
- **search_listings** - Zoeken naar Airbnb-properties op locatie en criteria
- **get_listing_details** - Gedetailleerde informatie opvragen over specifieke properties
- **check_availability** - Beschikbaarheid controleren voor specifieke data
- **get_reviews** - Reviews ophalen voor properties
- **get_host_info** - Informatie krijgen over eigenaren van properties

### Vereisten:

- **Node.js** geïnstalleerd op uw systeem
- **Internetverbinding** om het MCP-serverpakket te downloaden
- **NPX** beschikbaar (wordt meegeleverd met Node.js)

### De verbinding testen:

U kunt de MCP-server handmatig testen door het volgende uit te voeren:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Dit zal de OpenBnB MCP-server downloaden en starten, waarmee Semantic Kernel vervolgens verbinding maakt voor echte Airbnb-data.


## De Agent uitvoeren met OpenBnB MCP Server

Nu zullen we de AI Agent uitvoeren die verbinding maakt met de OpenBnB MCP-server om te zoeken naar echte Airbnb-accommodaties in Stockholm voor 2 volwassenen en 1 kind. Voel je vrij om de lijst `user_inputs` aan te passen om de zoekcriteria te wijzigen.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Samenvatting
Gefeliciteerd! Je hebt succesvol een AI-agent gebouwd die integreert met het zoeken naar accommodaties in de echte wereld via het Model Context Protocol (MCP):

## Gebruikte Technologieën:
- Semantic Kernel - Voor het bouwen van intelligente agents met Azure OpenAI
- Microsoft Foundry - Voor LLM-mogelijkheden en chat-afronding
- MCP (Model Context Protocol) - Voor gestandaardiseerde toolintegratie
- OpenBnB MCP Server - Voor echte Airbnb-zoekfunctionaliteit
- Node.js/NPX - Voor het uitvoeren van de externe MCP-server

## Wat Je Hebt Geleerd:
- MCP-integratie: Semantic Kernel-agents verbinden met externe MCP-servers
- Real-time Data Toegang: Werkelijke Airbnb-woningen doorzoeken via live API's
- Protocol Communicatie: Gebruik van stdio-communicatie tussen agent en MCP-server
- Functie Ontdekking: Automatisch beschikbare functies van MCP-servers ontdekken
- Streaming Antwoorden: Functie-aanroepen in real-time vastleggen en loggen
- HTML Rendering: Agent-antwoorden opmaken met gestileerde tabellen en interactieve weergaven

## Volgende Stappen:
- Extra MCP-servers integreren (weer, vluchten, restaurants)
- Een multi-agent systeem bouwen dat MCP- en A2A-protocollen combineert
- Eigen MCP-servers maken voor je eigen databronnen
- Persistente conversatiegeheugen implementeren over sessies heen
- De agent implementeren op Azure Functions met MCP-server orchestratie
- Gebruikersauthenticatie en boekingsmogelijkheden toevoegen


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsdienst [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, dient u zich ervan bewust te zijn dat automatische vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in de oorspronkelijke taal moet als de gezaghebbende bron worden beschouwd. Voor kritieke informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor eventuele misverstanden of foutieve interpretaties die voortvloeien uit het gebruik van deze vertaling.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
